In [27]:
!pip install tesseract 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import pymongo
import magic
import pdfplumber
from PIL import Image
import pytesseract
# from docx import Document
from io import BytesIO
import logging
import json
import pandas as pd

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# MongoDB connection
db = client["Nexus360"]
collection = db["Uploaded_Documents"]

# Mime type based approach to handle different types of files
def handle_pdf(file_data):
    with pdfplumber.open(BytesIO(file_data)) as pdf:
        text = ''.join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text

def handle_text(file_data):
    return file_data.decode('utf-8')
def handle_csv(file_data):
    csv_data = pd.read_csv(BytesIO(file_data))
    return csv_data.to_string(index=False)

# def handle_image(file_data):
#     image = Image.open(BytesIO(file_data))
#     text = pytesseract.image_to_string(image)
#     return text

# def handle_docx(file_data):
#     doc = Document(BytesIO(file_data))
#     text = '\n'.join(paragraph.text for paragraph in doc.paragraphs)
#     return text

# Function to process files
def process_files():
    mime = magic.Magic(mime=True)
    

    
    for document in collection.find():
        logging.info(f"Processing document ID: {document['_id']}")
        if 'FileData' in document:
            file_data = document['FileData']
            mime_type = mime.from_buffer(file_data)
            
            text = None
            if 'pdf' in mime_type:
                text = handle_pdf(file_data)
            elif 'plain' in mime_type:
                text = handle_text(file_data)
            elif 'csv' in mime_type:
                text = handle_csv(file_data)
            # elif 'image' in mime_type:
            #     text = handle_image(file_data)
            # elif 'officedocument' in mime_type:
            #     text = handle_docx(file_data)
            metadata = {
                "DocumentID": document.get("DocumentID", ""),
                "FileName": document.get("FileName", ""),
                "UploadedBy": document.get("UploadedBy", ""),
                "SpaceName": document.get("SpaceName", ""),
                "SpaceID": document.get("SpaceID", ""),
                "CreatedDate": document.get("CreatedDate", ""),
                "UpdatedDate": document.get("UpdatedDate", "")
                }
            result = {
                "text": text,
                "metadata": metadata
                }
            
            
            if text:
                # Save to a file
                with open(f"{document['_id']}.json", "w", encoding='utf-8') as f:
                    json.dump(result, f, indent=4, default=str)

if __name__ == "__main__":
    process_files()


2024-04-15 13:46:51,042 - INFO - Processing document ID: 660fce077ab5233c0a3638bd
2024-04-15 13:46:51,074 - INFO - Processing document ID: 6617c53981a80fc7e0b26611
2024-04-15 13:46:51,091 - INFO - Processing document ID: 6617c65de7aa7c816d9f859d
2024-04-15 13:46:51,110 - INFO - Processing document ID: 6617d1e1fd1364c41db7d235
2024-04-15 13:46:51,125 - INFO - Processing document ID: 6618d64d0f10d40b51e525bd
2024-04-15 13:46:54,430 - INFO - Processing document ID: 6618d6b70f10d40b51e525be
2024-04-15 13:46:56,877 - INFO - Processing document ID: 6618d7230f10d40b51e525bf
2024-04-15 13:46:59,402 - INFO - Processing document ID: 6618d7490f10d40b51e525c0
2024-04-15 13:47:01,788 - INFO - Processing document ID: 6618d7570f10d40b51e525c1
2024-04-15 13:47:03,577 - INFO - Processing document ID: 6618e0e05cd16fd6a4b67545
2024-04-15 13:47:06,551 - INFO - Processing document ID: 6618e14d5cd16fd6a4b67548
2024-04-15 13:47:10,464 - INFO - Processing document ID: 6618e1865cd16fd6a4b6754b
2024-04-15 13:47